<a href="https://colab.research.google.com/github/vkrisvasan/GenAIAgent/blob/main/AgentEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#build and deploy an agent that uses the Gemini Pro model, VertexAI Reasoning Engine, Python functions as tools, and LangChain for orchestration
#The Agent retrieves the exchange rate between two currencies on a specified date

#Gitaccount: vkrisvasan Repo: GenAIAgent Code: AgentEx1.ipynb
#Simple Agent assist implemented by
  #1. creating a project in google vertexai https://console.cloud.google.com/vertex-ai [note the projectID]
  #2. create storage bucket in https://console.cloud.google.com/storage/browser - use  "us-central1" location [Note location and bucket ID]


In [ ]:
!pip install --upgrade --quiet \
    google-cloud-aiplatform==1.51.0 \
    langchain==0.1.20 \
    langchain-google-vertexai==1.0.3 \
    cloudpickle==3.0.0 \
    pydantic==2.7.1 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os
import getpass
credential_names = ["GCP_PROJECT_ID", "LOCATION", "STAGING_BUCKET"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [ ]:
PROJECT_ID = os.environ["GCP_PROJECT_ID"]
LOCATION = os.environ["LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]
STAGING_BUCKET_FULL = "gs://{}".format(STAGING_BUCKET)


In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET_FULL)

In [ ]:
import requests
from vertexai.preview import reasoning_engines

In [ ]:
model = "gemini-1.5-pro-preview-0409"

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="INR")

In [ ]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
agent.query(input="What's the exchange rate from US dollars to Indian currency today?")

In [ ]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [ ]:
#Create vertexai Managed Runtime (Reasoning Engine): Effortlessly deploy and scale your application using Vertex AI’s secure and reliable infrastructure.
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "requests",
    ],
)

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Indian currency today?"
)